In [1]:
import os
import pandas as pd 
import json 

from src.textual_features import exact_keyword_search

# Exact keyword search for paragraphs from BauNVO & BauGB

## Prepare data

- Change the folder path in the code block below to read in the data.
- Specify the relevant column names. The function that is used in the following expects the input data frame to have (at least) two columns, i.e., one id and one content column. Here, the columns are called filename and content. If named differently, change the column names in the code below.


In [2]:
# specify file path
INPUT_FILE_PATH = os.path.join("data", "proc", "building_plans_sample", "bp_text.json")
OUTPUT_FILE_PATH = os.path.join("data", "nrw", "bplan", "features", "keywords", "exact_search", "exact_search.csv")

# specify relevant column names
ID_COLUMN='filename'
TEXT_COLUMN='content'

# read in data
input_df = pd.read_json(INPUT_FILE_PATH)


In [3]:
input_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315 entries, 0 to 314
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  315 non-null    object
 1   content   94 non-null     object
 2   metadata  315 non-null    object
dtypes: object(3)
memory usage: 7.5+ KB


In [4]:
input_df

,filename,content,metadata
0,10530_1.pdf,None,"{'Content-Type': 'application/octet-stream', '..."
1,1234_1.pdf,None,"{'Content-Type': 'application/octet-stream', '..."
2,1234_2.pdf,None,"{'Content-Type': 'application/octet-stream', '..."
3,1234_3.pdf,None,"{'Content-Type': 'application/octet-stream', '..."
4,1234_4.pdf,None,"{'Content-Type': 'application/octet-stream', '..."
...,...,...,...
310,81236_6.pdf,None,"{'Content-Type': 'application/octet-stream', '..."
311,81236_7.pdf,None,"{'Content-Type': 'application/octet-stream', '..."
312,81236_8.pdf,None,"{'Content-Type': 'application/octet-stream', '..."
313,81236_9.pdf,None,"{'Content-Type': 'application/octet-stream', '..."


## Define keyword dictionary

Keywords are specified in a separate json file to apply the exact keyword search more easily to different sets of keywords, simply by reading in the relevant dictionary. The dictionary is structured so that each keyword category (e.g. baunvo-1) can contain one or more keywords to consider the category covered (e.g., "§1 baunvo", "1 baunvo", or "allgemeine vorschriften für bauflächen und baugebiete").


In [5]:
with open('dictionaries/keyword_dict_baunvo.json') as f:
    BAUNVO_KEYWORDS = json.load(f)


## Apply function

Exact keyword matching based on input dictionary, returns df showing which keyword appeared in each pdf per category.

In [6]:
result_df = exact_keyword_search.search_df_for_keywords(input_df=input_df,
                                   text_column_name=TEXT_COLUMN,
                                   id_column_name=ID_COLUMN,
                                   keyword_dict=BAUNVO_KEYWORDS)

We can explore the results by seeing the output dataframe:

In [7]:
result_df.head()

,filename,baunvo-1,baunvo-2,baunvo-3,baunvo-4,baunvo-4a,baunvo-5,baunvo-5a,baunvo-6,baunvo-6a,...,baunvo-14,baunvo-15,baunvo-16,baunvo-17,baunvo-18,baunvo-19,baunvo-20,baunvo-21,baunvo-21a,13b
0,10530_1.pdf,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,1234_1.pdf,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,1234_2.pdf,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,1234_3.pdf,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,1234_4.pdf,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


And inspect keyword coverage across all files.

In [8]:
result_df.count()

filename      315
baunvo-1       12
baunvo-2       12
baunvo-3       11
baunvo-4        9
baunvo-4a       0
baunvo-5        7
baunvo-5a       0
baunvo-6        7
baunvo-6a       1
baunvo-7       10
baunvo-8        4
baunvo-9        2
baunvo-10       4
baunvo-11       1
baunvo-12      12
baunvo-13       2
baunvo-13a      1
baunvo-14       7
baunvo-15       3
baunvo-16       1
baunvo-17       1
baunvo-18       6
baunvo-19      14
baunvo-20       7
baunvo-21       0
baunvo-21a     13
13b             0
dtype: int64

In [12]:
input_df.dropna(subset = ['content'])['metadata'][10]

{'Content-Type': ['application/pdf', 'image/jpeg'],
 'X-Parsed-By': ['org.apache.tika.parser.DefaultParser',
  'org.apache.tika.parser.pdf.PDFParser',
  ['org.apache.tika.parser.DefaultParser',
   'org.apache.tika.parser.jpeg.JpegParser']],
 'X-TIKA:content_handler': 'ToTextContentHandler',
 'X-TIKA:embedded_depth': ['0', '1'],
 'X-TIKA:parse_time_millis': ['588', '16'],
 'access_permission:assemble_document': 'true',
 'access_permission:can_modify': 'true',
 'access_permission:can_print': 'true',
 'access_permission:can_print_degraded': 'true',
 'access_permission:extract_content': 'true',
 'access_permission:extract_for_accessibility': 'true',
 'access_permission:fill_in_form': 'true',
 'access_permission:modify_annotations': 'true',
 'dc:format': 'application/pdf; version=1.3',
 'pdf:PDFVersion': '1.3',
 'pdf:charsPerPage': '954',
 'pdf:docinfo:producer': 'PyPDF2',
 'pdf:encrypted': 'false',
 'pdf:hasMarkedContent': 'false',
 'pdf:hasXFA': 'false',
 'pdf:hasXMP': ['false', 'false'],